In [6]:
import pandas as pd
import json

In [15]:
df = pd.read_json('albums_0-999.json')

In [19]:
df = df.T
df.head()

,album_type,artists,available_markets,copyrights,external_ids,external_urls,genres,href,id,images,label,name,popularity,release_date,release_date_precision,total_tracks,tracks,type,uri
6vV5UrXcfyQD1wu4Qo2I9K,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",[{'text': '© 2005 Atlantic Recording Corporati...,{'upc': '075678377969'},{'spotify': 'https://open.spotify.com/album/6v...,[],https://api.spotify.com/v1/albums/6vV5UrXcfyQD...,6vV5UrXcfyQD1wu4Qo2I9K,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Atlantic Records/ATG,The Cookbook,64,2005-07-04,day,16,{'href': 'https://api.spotify.com/v1/albums/6v...,album,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K
0z7pVBGOD7HCIB7S8eLkLI,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...","[{'text': '(P) 2003 Zomba Recording LLC', 'typ...",{'upc': '828765374828'},{'spotify': 'https://open.spotify.com/album/0z...,[],https://api.spotify.com/v1/albums/0z7pVBGOD7HC...,0z7pVBGOD7HCIB7S8eLkLI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Jive,In The Zone,80,2003-11-13,day,13,{'href': 'https://api.spotify.com/v1/albums/0z...,album,spotify:album:0z7pVBGOD7HCIB7S8eLkLI
25hVFAxTlDvXbx2X2QkUkE,album,[{'external_urls': {'spotify': 'https://open.s...,"[CH, LI]","[{'text': '(P) 2003 Sony Music Entertainment',...",{'upc': '886443814448'},{'spotify': 'https://open.spotify.com/album/25...,[],https://api.spotify.com/v1/albums/25hVFAxTlDvX...,25hVFAxTlDvXbx2X2QkUkE,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Columbia,Dangerously In Love (Alben für die Ewigkeit),17,2003-06-23,day,16,{'href': 'https://api.spotify.com/v1/albums/25...,album,spotify:album:25hVFAxTlDvXbx2X2QkUkE
6QPkyl04rXwTGlGlcYaRoW,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...","[{'text': '(P) 2002 Zomba Recording LLC', 'typ...",{'upc': '012414182326'},{'spotify': 'https://open.spotify.com/album/6Q...,[],https://api.spotify.com/v1/albums/6QPkyl04rXwT...,6QPkyl04rXwTGlGlcYaRoW,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Jive,Justified,80,2002-11-04,day,13,{'href': 'https://api.spotify.com/v1/albums/6Q...,album,spotify:album:6QPkyl04rXwTGlGlcYaRoW
6NmFmPX56pcLBOFMhIiKvF,album,[{'external_urls': {'spotify': 'https://open.s...,[],"[{'text': '© 2001 MCA Records Inc.', 'type': '...",{'upc': '00008811256524'},{'spotify': 'https://open.spotify.com/album/6N...,[],https://api.spotify.com/v1/albums/6NmFmPX56pcL...,6NmFmPX56pcLBOFMhIiKvF,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Geffen,Hot Shot (International Version #2),5,2000,year,15,{'href': 'https://api.spotify.com/v1/albums/6N...,album,spotify:album:6NmFmPX56pcLBOFMhIiKvF


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19261 entries, 6vV5UrXcfyQD1wu4Qo2I9K to 7D6Y19tjm4DQNch39FeWpO
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   album_type              18478 non-null  object
 1   artists                 18478 non-null  object
 2   available_markets       18478 non-null  object
 3   copyrights              18478 non-null  object
 4   external_ids            18478 non-null  object
 5   external_urls           18478 non-null  object
 6   genres                  18478 non-null  object
 7   href                    18478 non-null  object
 8   id                      18478 non-null  object
 9   images                  18478 non-null  object
 10  label                   18469 non-null  object
 11  name                    18478 non-null  object
 12  popularity              18478 non-null  object
 13  release_date            18478 non-null  object
 14  release_date_precisio

In [12]:
print(df)

   collaborative description  \
0          False               

                                                href                      id  \
0  https://api.spotify.com/v1/playlists/0bw8158RM...  0bw8158RM7roegphH1GYCx   

                                              images     name primary_color  \
0  [{'height': 640, 'url': 'https://mosaic.scdn.c...  Vetting          None   

   public                       snapshot_id      type  ...    owner.id  \
0    True  AAABIJSLYfVS0G/X1DWMDI3nNNKm6xlu  playlist  ...  1225630715   

  owner.type                owner.uri  \
0       user  spotify:user:1225630715   

                                         tracks.href  \
0  https://api.spotify.com/v1/playlists/0bw8158RM...   

                                        tracks.items tracks.limit  \
0  [{'added_at': '2017-06-23T23:21:59Z', 'added_b...          100   

                                         tracks.next tracks.offset  \
0  https://api.spotify.com/v1/playlists/0bw8158RM...         